In [1]:

import sys
from pathlib import Path

project_root = Path.cwd()
while not (project_root / 'config.py').exists() and project_root != project_root.parent:
    project_root = project_root.parent

SRC_DIR = project_root / 'src'
sys.path.insert(0, str(project_root))
sys.path.insert(0, str(SRC_DIR))


In [2]:

import os
import geopandas as gpd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from matplotlib.backends.backend_pdf import PdfPages
from datetime import datetime

from config import SHAPEFILES_DIR, EXPORT_FOLDER
from gpkg_service import list_gpkg_files, list_gpkg_layers, load_gpkg_layer


In [3]:

plot_output = widgets.Output(layout={'justify_content': 'center'})
preview_output = widgets.Output()
export_output = widgets.Output()

gpkg_files = list_gpkg_files(SHAPEFILES_DIR)
gpkg_selector = widgets.Dropdown(
    options=[(f.name, f) for f in gpkg_files],
    description='GPKG:',
    layout=widgets.Layout(width='320px')
)

layer_selector = widgets.Dropdown(
    options=[],
    description='Layer:',
    layout=widgets.Layout(width='320px')
)

sampling_dropdown = widgets.Dropdown(
    options=[('10%', 0.10), ('25%', 0.25), ('50%', 0.50), ('100%', 1.0)],
    value=1.0,
    description='Sample:',
    layout=widgets.Layout(width='180px')
)

export_format_dropdown = widgets.Dropdown(
    options=['.gpkg', '.geojson', '.pdf'],
    value='.gpkg',
    description='Export:',
    layout=widgets.Layout(width='180px')
)

export_button = widgets.Button(description='Export sample', layout=widgets.Layout(width='140px'))
title_html = widgets.HTML(value='', layout=widgets.Layout(justify_content='center'))


In [4]:

def update_layers(change):
    if gpkg_selector.value:
        layers = list_gpkg_layers(gpkg_selector.value)
        layer_selector.options = layers
        if 'buurten' in layers:
            layer_selector.value = 'buurten'
        elif layers:
            layer_selector.value = layers[0]


In [5]:

def update_plot(*args):
    plot_output.clear_output()
    preview_output.clear_output()
    export_output.clear_output()
    try:
        gpkg_path = gpkg_selector.value
        layer = layer_selector.value
        gdf = load_gpkg_layer(gpkg_path, layer)
        frac = sampling_dropdown.value
        global sampled_gdf
        sampled_gdf = gdf.sample(frac=frac, random_state=1).reset_index(drop=True)
        title_html.value = f"<h4 style='text-align:center;'>📦 {gpkg_selector.label} | 🧭 Layer: {layer} | Sample: {int(frac*100)}%</h4><br>"
        meta_info.value = f'''
        <div style="text-align:left;">
            <b>GPKG-bestand:</b> {gpkg_selector.label}<br>
            <b>Layer:</b> {layer}<br>
            <b>Sample:</b> {int(frac * 100)}%
        </div>
        '''
        with plot_output:
            fig, ax = plt.subplots(figsize=(6, 6))
            sampled_gdf.plot(ax=ax)
            ax.set_xlabel('')
            ax.set_ylabel('')
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)
            plt.tight_layout()
            plt.show()
        with preview_output:
            print('🧾 Sample preview:')
            display(sampled_gdf.head(3))
            print(f'Shape: {sampled_gdf.shape[0]} rows × {sampled_gdf.shape[1]} columns')
    except Exception as e:
        with plot_output:
            print('⚠️ Plotfout:', e)


In [6]:

def export_sample(b):
    export_output.clear_output()
    try:
        ext = export_format_dropdown.value
        fname_base = f"sample_{layer_selector.value}_{int(sampling_dropdown.value*100)}"
        data_path = os.path.join(EXPORT_FOLDER, f"{fname_base}{ext}")
        pdf_path = os.path.join(EXPORT_FOLDER, f"{fname_base}.pdf")
        sampled_gdf.to_file(data_path, driver='GeoJSON' if ext == '.geojson' else 'GPKG')

        with PdfPages(pdf_path) as pdf:
            fig = plt.figure(figsize=(8.27, 11.69))  # A4 staand

            # Titel, bestandsnaam, filterinfo direct op het canvas
            title = "GPKG Export"
            bestand = f"Bestand: {gpkg_selector.label}"
            filterinfo = f"Layer: {layer_selector.value} | Sample: {int(sampling_dropdown.value*100)}%"

            fig.text(0.5, 0.95, title, ha='center', fontsize=14, weight='bold')
            fig.text(0.5, 0.92, bestand, ha='center', fontsize=10)
            fig.text(0.5, 0.89, filterinfo, ha='center', fontsize=10)

            # Eén enkel subplot voor geoplot (geen dubbele assen)
            ax = fig.add_subplot(111, position=[0.1, 0.1, 0.8, 0.75])
            sampled_gdf.plot(ax=ax)
            ax.set_axis_off()

            pdf.savefig(fig, bbox_inches='tight')
            plt.close(fig)

        with export_output:
            print(f'✅ Data geëxporteerd naar: {data_path}')
            print(f'📄 PDF gegenereerd: {pdf_path}')
    except Exception as e:
        with export_output:
            print(f'❌ Export naar PDF mislukt: {e}')


In [7]:

gpkg_selector.observe(update_layers, names='value')
gpkg_selector.observe(update_plot, names='value')
layer_selector.observe(update_plot, names='value')
sampling_dropdown.observe(update_plot, names='value')
export_button.on_click(export_sample)

export_controls = widgets.VBox([
    sampling_dropdown,
    widgets.HBox([export_format_dropdown, export_button], layout=widgets.Layout(gap='10px'))
])

control_block = widgets.VBox([
    gpkg_selector,
    layer_selector,
    export_controls,
    widgets.HTML('<br>'),
    export_output
])

meta_info = widgets.HTML(
    value=f'''
    <div style="text-align:left;">
        <b>GPKG-bestand:</b> {gpkg_selector.label}<br>
        <b>Layer:</b> {layer_selector.value}<br>
        <b>Sample:</b> {int(sampling_dropdown.value * 100)}%
    </div>
    ''',
    layout=widgets.Layout(width='300px')
)

plot_and_info = widgets.VBox([
    plot_output,
    widgets.HTML('<br>'),
    meta_info
])

vh_layout = widgets.VBox([
    widgets.HTML('<br>'),
    widgets.HTML("<h3 style='text-align:center;'>GPKG Sample generator</h3>"),
    control_block,
    widgets.HTML('<br>'),
    plot_and_info,
    widgets.HTML('<br>'),
    preview_output
])

display(vh_layout)
update_layers(None)
update_plot()
